In [1]:
import os
import pandas as pd
import re
import ngrams

In [2]:
import fasttext
ft = fasttext.load_model('cc.en.300.bin')

In [20]:
import numpy as np
# 计算点积
vec1=ft.get_word_vector("trash")
vec2=ft.get_word_vector("garbage")
vec3=ft.get_word_vector("king")
def cosine_sim(vec1, vec2):
    dot_product = np.dot(vec1,vec2)
    # 计算向量的模长
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    # 计算余弦相似度
    cos_sim = dot_product / (magnitude_vec1 * magnitude_vec2)
    return cos_sim

print(cosine_sim(vec1,vec2))
print(cosine_sim(vec2,vec3))

0.8648424
0.15605485


In [ ]:
####回来写全局属性的数据结构
attributeList = []

In [13]:
#将属性名按照标点，分割成token
def snake_case_split(line):
    #Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
    #r表示非转义的原始字符串
    #由于正则表达式通常都包含反斜杠，所以你最好使用原始字符串来表示它们。模式元素(如 r'\t'，等价于 '\\t')匹配相应的特殊字符。
    #[...]用来表示一组字符,单独列出：[amk] 匹配 'a'，'m'或'k'
    line_split = re.split(r'[\s_]',line.strip())
    line_split = [line.strip() for line in line_split if len(line.strip())>0]
    return line_split


#将属性按照大写字母，分割成token
#"firstName"分成 ['First', 'Name']
def camel_case_split(str):
    if "a"<=str[0]<="z" :
        strTemp=str[0].upper()+str[1:]
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', strTemp)
    else :
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)


#将属性名先按照snakeCase拆分，再按照camelCase拆分
#最后按照语料库拆分
def attributeToToken(str):
    tokenList = []
    for snakeToken in snake_case_split(str):
        for camelToken in camel_case_split(snakeToken):
            camelToken = camelToken.lower()
            corpusTokenList = ngrams.segment2(camelToken) [1]
            tokenList = tokenList + corpusTokenList
    return tokenList

['machine', 'name', 'backend']


In [ ]:
class Attribute:
    def __init__(self, attributeName, originateFrom):
        self.attributeName = attributeName
        self.tokenList = attributeToToken(attributeName)
        #属性来自哪张表
        self.originateFrom = originateFrom

In [ ]:
for info in os.listdir('/usr/project/TC/example'):
    tableName = info[0:-4]#去掉.csv后缀
    domain = os.path.abspath('/usr/project/TC/example')#获取文件夹的路径
    info = os.path.join(domain,info)#将路径与文件名结合起来就是每个文件的完整路径
    df = pd.read_csv(info)
    for attributeName in list(df):
        x=Attribute(attributeName, tableName)
        attributeList.append(x)